In [19]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print("HIP version:", torch.version.hip)
print("Device count:", torch.cuda.device_count())
print("Device 0:", torch.cuda.get_device_name(0))

Using device: cuda
HIP version: 6.2.41133-dd7f95766
Device count: 1
Device 0: AMD Radeon RX 6800


In [20]:
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.tensorboard import SummaryWriter

In [21]:
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [22]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

val_size = 5000
test_size = len(test_data) - val_size

val_data, test_data = random_split(test_data, [val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

print(f'Training dataset: {len(train_loader.dataset)}')
print(f'Validation dataset: {len(val_loader.dataset)}')
print(f'Test dataset: {len(test_loader.dataset)}')


Files already downloaded and verified
Files already downloaded and verified
Training dataset: 50000
Validation dataset: 5000
Test dataset: 5000


In [23]:
image, label = train_data[0]

In [24]:
image.size()

torch.Size([3, 32, 32])

In [25]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [26]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3)
        self.conv2 = nn.Conv2d(64, 128, 3)
        self.conv3 = nn.Conv2d(128, 256, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(256 * 2 * 2, 512)  
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [27]:
class MyResnet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.backbone = resnet18(weights=None)
        self.backbone.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.backbone.maxpool = nn.Identity()
        self.backbone.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        return self.backbone(x)


In [28]:
net = MyResnet()
net = net.to(device)

In [29]:
#net = NeuralNet().to(device)
loss_function = nn.CrossEntropyLoss()

num_of_epochs = 70

optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_of_epochs)

torch.manual_seed(42)

In [30]:
writer = SummaryWriter()

In [31]:

highest_val_accuracy = 0.0

for epoch in range(num_of_epochs):
    net.train()
    
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_function(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    net.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total

    if val_accuracy > highest_val_accuracy:
        highest_val_accuracy = val_accuracy

        checkpoint = {
            "epoch": epoch,
            "model_state": net.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "scheduler_state": scheduler.state_dict(),
            "config": {
                "conv1_kernel": 3,
                "remove_maxpool": True,
                "num_classes": 10
            },
            "best_val_accuracy": highest_val_accuracy
        }

        torch.save(checkpoint, "trained_model.pth")
    
    global_step = epoch + 1
    writer.add_scalar("Loss/Train", avg_train_loss, global_step)
    writer.add_scalar("Loss/Val", avg_val_loss, global_step)
    writer.add_scalar("Accuracy/Val", val_accuracy, global_step)
    writer.add_scalar("Params/LR", optimizer.param_groups[0]["lr"], global_step)

    print(
        f'Epoch ({epoch + 1}/{num_of_epochs}),'
        f'Train Loss: {avg_train_loss:.4f}, '
        f'Val Loss: {avg_val_loss:.4f}, '
        f'Val Accuracy: {val_accuracy:.2f}%'
        )

    scheduler.step()

writer.flush()

print(f'Highest Validation Accuracy: {highest_val_accuracy:.2f}%')

Epoch (1/70),Train Loss: 1.9152, Val Loss: 1.5768, Val Accuracy: 41.56%
Epoch (2/70),Train Loss: 1.5104, Val Loss: 1.5412, Val Accuracy: 46.16%
Epoch (3/70),Train Loss: 1.2502, Val Loss: 1.3366, Val Accuracy: 55.26%
Epoch (4/70),Train Loss: 1.0853, Val Loss: 1.1858, Val Accuracy: 59.60%
Epoch (5/70),Train Loss: 0.9885, Val Loss: 1.1810, Val Accuracy: 59.30%
Epoch (6/70),Train Loss: 0.9375, Val Loss: 1.1498, Val Accuracy: 60.28%
Epoch (7/70),Train Loss: 0.9055, Val Loss: 1.0072, Val Accuracy: 66.22%
Epoch (8/70),Train Loss: 0.8787, Val Loss: 1.1707, Val Accuracy: 61.78%
Epoch (9/70),Train Loss: 0.8650, Val Loss: 1.3965, Val Accuracy: 56.92%
Epoch (10/70),Train Loss: 0.8497, Val Loss: 0.9806, Val Accuracy: 66.14%
Epoch (11/70),Train Loss: 0.8362, Val Loss: 0.9193, Val Accuracy: 69.94%
Epoch (12/70),Train Loss: 0.8283, Val Loss: 1.2503, Val Accuracy: 60.56%
Epoch (13/70),Train Loss: 0.8268, Val Loss: 0.7832, Val Accuracy: 72.58%
Epoch (14/70),Train Loss: 0.8109, Val Loss: 1.2502, Val Accu

In [32]:
net = MyResnet()
checkpoint = torch.load("trained_model.pth")
net.load_state_dict(checkpoint["model_state"])

/tmp/ipykernel_65862/749520413.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("trained_model.pth")


<All keys matched successfully>

In [33]:
correct = 0
total = 0

net = net.to(device)
net.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 92.42%


In [34]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['data/tests/ship.jpg', 'data/tests/cat.jpg', 'data/tests/dog.jpg']
images = [load_image(path).to(device) for path in image_paths]

net.eval()
with torch.no_grad():
    for i, image in enumerate(images):
        outputs = net(image)
        _, predicted = torch.max(outputs, 1)
        print(f'Image: {image_paths[i]}, Predicted class: {class_names[predicted.item()]}')

Image: data/tests/ship.jpg, Predicted class: ship
Image: data/tests/cat.jpg, Predicted class: cat
Image: data/tests/dog.jpg, Predicted class: dog
